In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from scipy import interp
import os
from data_accuracy import *

%matplotlib inline

In [52]:
pd.concat([df_test_pred.drop([1,3], axis=1), df_test_pred[1].apply(pd.Series)], axis=1)

,0,2,0,1,2,3
0,00001.png,Compulsory Keep Left,378,497,441,557
1,00001.png,Cycle Crossing,986,394,1028,434
2,00001.png,Pedestrian Crossing,1026,318,1082,370
3,00023.png,Right Hand Curve,952,429,1014,496
4,00023.png,School Ahead,40,345,140,449
5,00023.png,Speed Limit 50,993,288,1059,368
6,00001.png,Compulsory Keep Left,378,497,441,557
7,00001.png,Cycle Crossing,986,394,1028,434
8,00001.png,Pedestrian Crossing,1026,318,1082,370
9,00023.png,Right Hand Curve,952,429,1014,496


# Loding the original annotated text file

In [6]:
df_r=pd.read_csv('/home/ce/Documents/py-faster-rcnn/Accuracy_Check/annotationTools/sampleResults/results29-90.csv',sep=';',header=None,dtype=str)

In [7]:
df_tr=pd.read_csv('/home/ce/Documents/py-faster-rcnn/Accuracy_Check/annotationTools/sampleResults/stop-split2.csv',sep=';',header=None,dtype=str)

In [4]:
test_file = raw_input("Enter the original annotated txt file with path: ")
df_test=pd.read_csv(test_file,sep=';',header=None,dtype=str)
print 'Number of files in which signs are annotated',len(np.unique(df_test[0]))

Enter the original annotated txt file with path: /home/ce/Documents/py-faster-rcnn/data/GTSDB/data/train_gtsdb.txt
Number of files in which signs are annotated 297


# Loading the predicted text file

In [3]:
test_file_pred = raw_input("Enter the predicted txt file with path: ")
df_test_pred=pd.read_csv(test_file_pred,sep=';',header=None) #output of the test files
print 'Number of files in which signs are detected',len(np.unique(df_test_pred[0]))

Enter the predicted txt file with path: /home/ce/Documents/py-faster-rcnn/output_mmi.txt
Number of files in which signs are detected 3


In [21]:
# Updating the BBox which are in float to int
df_test_pred[1]=df_test_pred[1].apply(lambda x:get_bbox(x))
print 'Bbox in float is updated to int'

Bbox in float is updated to int


In [54]:
df_test_pred = pd.concat([df_test_pred.drop([1,3], axis=1), df_test_pred[1].apply(pd.Series)], axis=1)

In [66]:
df_test_pred.columns=[1,2,3,4,5,6]

In [83]:
df_test_pred[1]+'_'+df_test_pred[0]

KeyError: 0

In [69]:
df_test_pred[df_test_pred[2]=='Compulsory Keep Left']

(25, 6)

In [80]:
df_test.head()

,0,1,2,3,4,5
0,00005,1172,164,1284,278,no overtaking
1,00006,926,350,989,414,speed limit 50
2,00007,825,406,864,445,no overtaking
3,00009,925,466,949,490,speed limit 70
4,00012,979,364,1034,418,speed limit 30


In [75]:
df_tr=df_tr[1:]

In [81]:
df_tr

,0,1,2,3,4,5,6,7,8,9
1,2014-04-24_10-59/frameAnnotations-cam1.avi_ann...,stop,914,332,990,410,training/2014-04-24_10-59/cam1.avi,3539,stop_1398812275.avi,36
2,2014-05-01_17-03/2/frameAnnotations-2.avi_anno...,stop,273,403,319,456,training/2014-05-01_17-03/2/2.avi,1674,stop_1404948446.avi,16
3,2014-05-01_10-40/frameAnnotations-cam1.avi_ann...,stop,964,474,1036,537,training/2014-05-01_10-40/cam1.avi,12701,stop_1398988503.avi,11
4,2014-07-10_12-00/1/frameAnnotations-1.avi_anno...,stop,840,367,903,431,training/2014-07-10_12-00/1/1.avi,1081,stop_1405043501.avi,36
5,2014-04-24_10-59/frameAnnotations-cam1.avi_ann...,stop,905,372,958,425,training/2014-04-24_10-59/cam1.avi,7913,stop_1398812648.avi,46
6,2014-05-01_17-03/1/frameAnnotations-1.avi_anno...,stop,963,301,1055,396,training/2014-05-01_17-03/1/1.avi,4932,stop_1404947715.avi,31
7,2014-05-01_11-40/frameAnnotations-cam1.avi_ann...,stop,750,446,792,487,training/2014-05-01_11-40/cam1.avi,670,stop_1398992997.avi,26
8,2014-04-24_11-43/frameAnnotations-cam1.avi_ann...,stop,827,423,874,474,training/2014-04-24_11-43/cam1.avi,12334,stop_1398984963.avi,31
9,2014-04-24_10-59/frameAnnotations-cam1.avi_ann...,stop,771,411,805,448,training/2014-04-24_10-59/cam1.avi,14248,stop_1398813081.avi,1
10,2014-07-11_13-47/frameAnnotations-cam1.avi_ann...,stop,855,407,900,451,training/2014-07-11_13-47/cam1.avi,716,stop_1405372278.avi,6


In [ ]:
print 'Merging duplicates...'
print len(df_test_pred)
df_test_pred.drop(merge_duplicates(df_test_pred),inplace=True)
print len(df_test_pred)

In [46]:
print 'Accuracy for detection....'

Accuracy for detection....


In [47]:
for d in [df_test_pred]:
    tp=0
    fp=0
    fn=0
    for f in np.unique(d[0]):
        d1=df_test[df_test[0]==f]
        
        d2=d[d[0]==f]      

        flag=np.ones(len(d1))
        for i in d2.iterrows():
            flag2=0
            i=np.array(i)[1]
            bbox1=get_bbox(i[1])
            start=0
            for j in d1.iterrows():
                j=np.array(j)[1]
                bbox2=get_bbox(list(j[1:5]))
                if get_overlap(bbox1,bbox2)>0.5:
                    tp+=1
                    flag[start]=0
                    flag2=1
                start+=1

            if flag2==0:
                fp+=1

        fn+=np.sum(flag)

    print 'tp',tp,'  fp', fp,'   fn',fn
    print 'precison',float(tp)/(tp+fp)
    print 'recall',float(tp)/(tp+fn)

IndexError: list index out of range

In [ ]:
fpr, tpr, thresholds = roc_curve(y[test], probas_[:, 1])
roc_auc = auc(fp, tp)
print 'Area under the curve: ' + str(roc_auc)

In [ ]:
print 'Accuracy for recognition...'

In [ ]:
list1=[]
score=0.5
for l in training_labels:
    tp=0
    fp=0
    fn=0
    counter=0
    counter1=0
    df_final1=df_test_pred[df_test_pred[2]==l]
    df_test1=df_final1[df_final1[3]>score].copy()  #use df_test or df
    for f in np.unique(df_test1[0]):
        d1=df_test[df_test[0]==f]
        #d2=df[df[0]==f]
        d2=df_test1[df_test1[0]==f]
        counter+=len(d1)
        flag=np.ones(len(d1))

        for i in d2.iterrows():
            flag1=0
            counter1+=1
            i=np.array(i)[1]
            bbox1=get_bbox(i[1])
            start=0
            for j in d1.iterrows():
                j=np.array(j)[1]
                bbox2=get_bbox(list(j[1:5]))
                if get_overlap(bbox1,bbox2)>0.5:
                    if (i[2]==j[5]) :
                        tp+=1
                        flag[start]=0
                        flag1=1

            if flag1==0:
                fp+=1

        fn+=int(np.sum(flag))

    if float(tp)/(tp+fn) <0.5:
        print counter, 'should be equal to tp+fn=',tp+fn
        print counter1, 'should be equal to tp+fp=',tp+fp
        print tp,fp,fn
        print l,score,'precison',float(tp)/(tp+fp),'recall',float(tp)/(tp+fn),'\n'
        list1.append(l)